In [1]:
import numpy
import numpy as np
import pandas
import pandas as pd
import os
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
#from sklearn.decomposition import PCA
#from sklearn.cluster import KMeans

inline_rc = dict(mpl.rcParams)

CARGAMOS LOS DATOS DE yelp_academic_dataset_review.json

In [2]:
reviews = []
with open('data/datosExperimento.json', encoding="ansi") as fl:
    i=0
    for review in fl:
        reviews.append(json.loads(review))
        i+=1
        if i + 1 > 5:
            break

df_review = pd.DataFrame(reviews)
print(df_review.head())

FileNotFoundError: [Errno 2] No such file or directory: 'data/datosExperimento.json'

CARGAMOS LOS DATOS DE yelp_academic_dataset_business.json

In [ ]:
business = []
with open('data/yelp_academic_dataset_business.json', encoding="ansi") as fl:
    for i, line in enumerate(fl):
        business.append(json.loads(line))

df_business = pd.DataFrame(business)
print(df_business.head())



ELIMINAMOS LAS COLUMNAS INNECESARIAS

COMPROBAMOS

In [ ]:
df_review = df_review.drop(["review_id", "user_id","business_id", "useful", "funny", "cool", "date","stars"], axis=1)
#df_business = df_business.drop(["name", "address", "city", "state", "postal_code", "latitude", "longitude", "stars", "review_count", "is_open", "attributes", "hours"], axis=1)

In [ ]:
print(df_review.head())
#print(df_business.head())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

vectorizer = TfidfVectorizer()
lemma = WordNetLemmatizer()

X = []
for r in df_review["text"]: #Por cada review
    txt = word_tokenize(r.lower()) #Separa la review en palabras
    newtxt = ""
    for w in txt: #Por cada palabra en txt
        l = lemma.lemmatize(w) #se hace lo de quitar mayusculas y quitar raice y eso
        newtxt += l+" "
    X.append(newtxt)

X = vectorizer.fit_transform(X)
print(X)

In [ ]:
X= X.toarray()
X=pd.DataFrame(X)
#X=np.array(X, dtype=object)
print(X)

In [ ]:
from sklearn.decomposition import PCA

#Antes de aplicar el método de clasificación utilizamos PCA para reducir el número de atributos
print('Dim originally: ',X.shape)
#Como vamos a representar gráficamente los clusters, nos quedaremos con los 2 atributos más imporantes
pca = PCA(n_components=2)
pca.fit(X)
# Cambio de base a dos dimensiones PCA
X = pca.transform(X)
print('Dim after PCA: ',X.shape)
print(X)


In [ ]:
#X = pd.DataFrame(X)
#print(X[0][0])

In [ ]:
class Distance_computation_grid(object):

#De aquí por ahora solo utilizamos compute distant y distance calculate para solamente crear la matriz de distancias
#Para simplificar un poco lo de juntar instancias hago la matriz entera llena

    def __init__(self):
        pass

    def compute_distance(self,samples):

        Distance_mat = np.zeros((len(samples),len(samples))) #Hace una matriz de 10000*10000
        for i in range(Distance_mat.shape[0]):
            for j in range(Distance_mat.shape[0]):
                if i!=j and i < j:#Solo calcula distancias en una mitad de la matriz
                    distancia = float(self.distance_calculate(samples[i],samples[j])) #Distancia entrela muestra i e j
                    Distance_mat[i,j] = distancia #Meto en la matriz la distancia do vece
                    Distance_mat[j,i] = distancia
                elif i ==j:
                    Distance_mat[i,j] = 10**4 #Pone un valor muy grande para que nunca salga como la distancia más pequeña entrela misma instancia
        return Distance_mat


    def distance_calculate(self,sample1,sample2):

        dist = []
        for i in range(len(sample1)):
            for j in range(len(sample2)):
                    dist.append(abs(sample1[i][0]-sample2[j][0])+abs(sample1[i][1]-sample2[j][1]))
        return np.min(dist)


#Esto no se usa para nada lo he dejado más que nada por si había algo que nos interesaba de aquí.

    def intersampledist(self,s1,s2):
        if str(type(s2[0]))!='<class \'list\'>':
            s2=[s2]
        if str(type(s1[0]))!='<class \'list\'>':
            s1=[s1]
        m = len(s1)
        n = len(s2)
        dist = []
        if n>=m:
            for i in range(n):
                for j in range(m):
                    if (len(s2[i])>=len(s1[j])) and str(type(s2[i][0])!='<class \'list\'>'):
                        dist.append(self.interclusterdist(s2[i],s1[j]))
                    else:
                        dist.append(abs(s1[i][0]-s2[j][0])+abs(s1[i][1]-s2[j][1]))
        else:
            for i in range(m):
                for j in range(n):
                    if (len(s1[i])>=len(s2[j])) and str(type(s1[i][0])!='<class \'list\'>'):
                        dist.append(self.interclusterdist(s1[i],s2[j]))
                    else:
                        dist.append(abs(s1[i][0]-s2[j][0])+abs(s1[i][1]-s2[j][1]))
        return min(dist)

    def centroide(self,sample):
        if(len(sample) == 1):
            return np.mean()
        else:
            sample1=sample[0]
            i=1
            while i < len(sample):
                dist.append(abs(s1[i][0]-s2[j][0])+abs(s1[i][1]-s2[j][1]))

        return np.mean(dist)

    def interclusterdist(self,cl,sample):
        if sample[0]!='<class \'list\'>':
            sample = [sample]
        dist   = []
        for i in range(len(cl)):
            for j in range(len(sample)):
                dist.append(np.linalg.norm(np.array(cl[i])-np.array(sample[j])))
        return min(dist)

In [ ]:
#Aquí básicamente se hace la matriz de distancias osea aquí no creo que e cambie nada
progression = [[i] for i in range(X.shape[0])]
samples     = [[list(X[i])] for i in range(X.shape[0])]
m = len(samples)
distcal  = Distance_computation_grid()

print('Sample size before clustering    :- ',m)
Distance_mat      = distcal.compute_distance(samples)
print(Distance_mat)
dist_mat = pd.DataFrame(Distance_mat)
print(dist_mat)

In [ ]:
#Aquí se busca la distancia más pequeña y se devuelven los índices
minimos=np.matrix(dist_mat).argmin(axis = 1).tolist() #coordenadas x del valor minimo de cada fila. En dataframe es x,y, es decir en elprimer [] van las filas y en el segun do [] van las columnas
print(minimos)#Aquí te imprime las coordenadas de cada fila para que veas que formato tiene
minn=dist_mat[minimos[0][0]][0] #Aquí simplemente guardo el minimo de la primera fila, los [][] en minimos es porque argmin devuelve un array de arrays
print(minn)
y=0 #la coordenada y del elemento más pequeño de la primera fila
x=minimos[0][0] #la coordenada x del elemento más pequeño de la primera fila
for i in range(1,len(minimos)-1): #Para cada minimo en la lista de minimos
    if dist_mat[minimos[i][0]][i] < minn: #Si el mínimo de esa fila es más pequeño que el minimo actual
        minn=dist_mat[minimos[i][0]][i] #Se guarda el nuevo minimo junto a sus coordenadas.
        x=minimos[i][0]
        y=i
print('---------------------------------')
print(x)
print(y)
print(minn)

In [ ]:
#Aquí lo que estuve haciendo es intentar hacer la primera iteración de la matriz que genere con los primerass 5 instancias de datosExperimentos
#Recomiendo ver https://youtu.be/YH0r47m0kFM
nodo1 = dist_mat[0] #En el nodo1 guardo la columna 0
nodo2 = dist_mat[4] #En el nodo2 guardo la columna 4 (son las coordenadas del mininmo mas que na)
print(nodo1)
print(nodo2)
print(len(dist_mat))
eliminado = False #Un booleano para borrar solo una vez el minimo ya que sale do veces la pareja de 10000 y el minimo
nuevaFila = [] #Aqui voy a guardar las distancias nuevas obtenidas por complete link, es decir, de cada pareja de distancias te quedas con la más grande
for i in range(0,len(dist_mat)): #Para cada pareja de distancias
    if((nodo1[i]==minn or nodo2[i]==minn) and not eliminado): #Aquí me ocupo de la pareja de 10000 y el minimo que sale como primera
        nuevaFila.append(float(10**4))
        eliminado = True
    elif((nodo1[i]==minn or nodo2[i]==minn) and eliminado): #Aqui como sale otra vez la pareja 10000 y el minimo no hago nada con ella igual se puede poner los if de otra manera no se me ocurria de ptra manera ayer la verdad
        print()
    elif(nodo1[i]>=nodo2[i]): #Aqui en los dos siguiientes ifs nos quedamos con las distancias mas grandes de las parejas de distancias
            nuevaFila.append(float(nodo1[i]))
    else:
            nuevaFila.append(float(nodo2[i]))
print(pd.DataFrame(nuevaFila)) #Convierto la lista de distancias en un dataframe para poner meterlo a la matriz de ditancias
dist_mat = dist_mat.drop([0,4] ,axis=1) #Quito las filas y columnas de las instancias que se han mergeado
dist_mat = dist_mat.drop([0,4] ,axis=0)

#dist_mat.insert(0,'(1,4)',nuevaFila[1:]) Aquí estan mis intento de meter la nueva fila y columna
#dist_mat.loc[len(dist_mat)] = nuevaFila
print(dist_mat)



In [ ]:

sample_ind_needed = np.where(Distance_mat==Distance_mat.min())[0]
value_to_add      = samples.pop(sample_ind_needed[1])
samples[sample_ind_needed[0]].append(value_to_add)

print('Cluster Node 1                   :-',progression[sample_ind_needed[0]])
print('Cluster Node 2                   :-',progression[sample_ind_needed[1]])

progression[sample_ind_needed[0]].append(progression[sample_ind_needed[1]])
progression[sample_ind_needed[0]] = [progression[sample_ind_needed[0]]]
v = progression.pop(sample_ind_needed[1])
m = len(samples)

print('Progression(Current Sample)      :-',progression)
print('Cluster attained                 :-',progression[sample_ind_needed[0]])
print('Sample size after clustering     :-',m)
print('\n')


In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
def hierarchical_clustering(data,linkage,no_of_clusters):
    #first step is to calculate the initial distance matrix
    #it consists distances from all the point to all the point
    color = ['r','g','b','y','c','m','k','w']
    initial_distances = pairwise_distances(data,metric='euclidean')
    #making all the diagonal elements infinity
    np.fill_diagonal(initial_distances,sys.maxsize)
    clusters = find_clusters(initial_distances,linkage)

    #plotting the clusters
    iteration_number = initial_distances.shape[0] - no_of_clusters
    clusters_to_plot = clusters[iteration_number]
    arr = np.unique(clusters_to_plot)

    indices_to_plot = []
    fig = plt.figure()
    fig.suptitle('Scatter Plot for clusters')
    ax = fig.add_subplot(1,1,1)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    for x in np.nditer(arr):
        indices_to_plot.append(np.where(clusters_to_plot==x))
    p=0

    print(clusters_to_plot)
    for i in range(0,len(indices_to_plot)):
        for j in np.nditer(indices_to_plot[i]):
               ax.scatter(data[j,0],data[j,1], c= color[p])
        p = p + 1

    plt.show()


def find_clusters(input,linkage):
    clusters = {}
    row_index = -1
    col_index = -1
    array = []


    for n in range(input.shape[0]):
        array.append(n)

    clusters[0] = array.copy()

    #finding minimum value from the distance matrix
    #note that this loop will always return minimum value from bottom triangle of matrix
    for k in range(1, input.shape[0]):
        min_val = sys.maxsize

        for i in range(0, input.shape[0]):
            for j in range(0, input.shape[1]):
                if(input[i][j]<=min_val):
                    min_val = input[i][j]
                    row_index = i
                    col_index = j

        #once we find the minimum value, we need to update the distance matrix
        #updating the matrix by calculating the new distances from the cluster to all points

        #for Single Linkage
        if(linkage == "single" or linkage =="Single"):
            for i in range(0,input.shape[0]):
                if(i != col_index):
                    #we calculate the distance of every data point from newly formed cluster and update the matrix.
                    temp = min(input[col_index][i],input[row_index][i])
                    #we update the matrix symmetrically as our distance matrix should always be symmetric
                    input[col_index][i] = temp
                    input[i][col_index] = temp
        #for Complete Linkage
        elif(linkage=="Complete" or linkage == "complete"):
             for i in range(0,input.shape[0]):
                if(i != col_index and i!=row_index):
                    temp = min(input[col_index][i],input[row_index][i])
                    input[col_index][i] = temp
                    input[i][col_index] = temp
        #for Average Linkage
        elif(linkage=="Average" or linkage == "average"):
             for i in range(0,input.shape[0]):
                if(i != col_index and i!=row_index):
                    temp = (input[col_index][i]+input[row_index][i])/2
                    input[col_index][i] = temp
                    input[i][col_index] = temp

        elif(linkage=="Centroid" or linkage =="centroid"):
            for i in range(0,input.shape[0]):
                if(i!=col_index and i!=row_index):
                    dist_centroid = cal_dist_from_centroid(i,row_index,col_index)
                    input[col_index][i] = dist_centroid
                    input[i][col_index] = dist_centroid

        #set the rows and columns for the cluster with higher index i.e. the row index to infinity
        #Set input[row_index][for_all_i] = infinity
        #set input[for_all_i][row_index] = infinity
        for i in range (0,input.shape[0]):
            input[row_index][i] = sys.maxsize
            input[i][row_index] = sys.maxsize

        #Manipulating the dictionary to keep track of cluster formation in each step
        #if k=0,then all datapoints are clusters

        minimum = min(row_index,col_index)
        maximum = max(row_index,col_index)
        for n in range(len(array)):
            if(array[n]==maximum):
                array[n] = minimum
        clusters[k] = array.copy()

    return clusters
clusters = hierarchical_clustering(X,1,2)
print(clusters)

In [ ]:
# Import the f
print(clusters)

In [ ]:
cluster and linkage functions
from scipy.cluster.hierarchy import fcluster, linkage

# Use the linkage() function
distance_matrix = linkage(X, method = 'ward', metric = 'euclidean')

In [ ]:
# Import the dendrogram function
from scipy.cluster.hierarchy import dendrogram

# Create a dendrogram
dn = dendrogram(distance_matrix)

# Display the dendogram
plt.show()